In [1]:
import pandas as pd
import sklearn
import math
import numpy as np

In [2]:
filename = "70K_IlanDatasi_wLi_230120.xlsx"
#filename = "test_1.xlsx"
sheetName = "Sheet1"
print("Reading data...")
dfFirst = pd.read_excel(filename, sheet_name = sheetName)
print("Reading done!")

Reading data...
Reading done!


In [3]:
print("Reading cities")
f = open("il.txt", encoding="utf-8")
cities = f.read();
cities = cities.split("\n")
f.close();
print("City reading is done.")

Reading cities
City reading is done.


In [9]:
print("Reading countries")
f = open("ülkeler.txt", encoding="utf-8")
countries = f.read();
countries = countries.split("\n")
f.close();
print("Country reading is done.")

Reading countries
Country reading is done.


In [4]:
education_keywords = ["Mezun", "Üniversite", "Lise", "Öğrenci", "Doktora", "Meslek", "Yüksekokul"]

In [5]:
#Function to apply 
def check_city_allowed(cities, x):
    #First requirement
    res1 = False;
    for city in cities:
        res1 = res1 or city in x or city.lower() in x.lower() or city.upper() in x.upper()
    #Second requirement
    res2 = ("türkiye" in x.lower())
    #Third requirement
    res3 = (x.strip() == "")
    return res1 or res2 or res3

In [6]:
def contains_country(countries, text):
    for country in countries:
        if country.lower() in text:
            return country
    return "";

def look_for_country(df, countries):
    rows_to_delete = []
    for index, row in df.iterrows():
        #If city is blank, try to fill it with the counrty info from the text.
        #If not, delete that entry.
        if row.iller == " ":
            country = contains_country(countries, row.metinTemiz.lower())
            if country != "":
                df.at[index, 'iller'] = country
            else:
                rows_to_delete.append(index)
    if(len(rows_to_delete) != 0):
        df.drop(rows_to_delete, inplace=True)
    return df;

In [7]:
def validate_education_status(keywords, x):
    for keyword in keywords:
        if keyword.lower() in x.lower():
            return True
    return False

KURALLAR
1. Numeric olması gereken alanlar (jobrefno, Tecrube, maxTecrube), eğer numeric değilse silinecek.
2. Tecrübe puanı 40 üzeri olanlar 0 olarak değiştirilecek.
3. Max Tecrube değişiklikleri:

	a. 0 ve 20 eşit ve üzeri olanlar 99 olarak değişecek
    
	b. (MaxTecrube - Tecrube) <= Tecrube --> MaxTecrube = MaxTecrube + 3 şeklinde değiştir.
4. İller alanı için saçma girdileri sil.
		Saçma Girdiler = {Boş olmayanlar, 81 ilden birini içermeyenler, Tüm Türkiye yazmayanlar}
5. İller alanında boş olan veriler için temiz metin içerisinden farklı ülke bilgilerini
	 kontrol et. Eğer metin içerisinde farklı bir ülke var ise iller alanını o ülke ile değiştir.
6. Eğitim durumu bilgisi boş olan veriler silinecek
7. Eğitim durumu bilgisinde keywordlerden birini içermeyen veriler sinilecek
	keywords = ["Mezun", "Üniversite", "Lise", "Öğrenci", "Doktora", "Meslek", "Yüksekokulu"]

In [12]:
df = dfFirst.copy()
print(df.size//9)
#Fill NaN values as blanks
df = df.fillna(' ')
#Apply Rule 1
df = df[df.jobrefno.apply(lambda x : False if type(x) == str else True)]
df = df[df.Tecrube.apply(lambda x : False if type(x) == str else True)]
df = df[df.maxTecrube.apply(lambda x : False if type(x) == str else True)]
#Apply Rule 2
df.Tecrube = df.Tecrube.apply(lambda x : 0 if x > 40 else x)
#Apply Rule 3
df.maxTecrube = df.maxTecrube.apply(lambda x : 99 if x == 0 or x >= 20 else x)
df.maxTecrube = df.apply(lambda x : x.maxTecrube+3 if x.maxTecrube-x.Tecrube <= x.Tecrube else x.maxTecrube, axis=1)
# Apply Rule 4
df = df[df.iller.apply(lambda x : check_city_allowed(cities, x))]
print(df.size//9)
#Apply Rule 5
df = look_for_country(df, countries);
print(df.size//9)
#Apply Rule 6 and 7
df = df[df.EgitimDurumuUzun.apply(lambda x : x != " ")]
df = df[df.EgitimDurumuUzun.apply(lambda x : validate_education_status(education_keywords, x))]
print(df.size//9)

69569
69565
69377
69377


In [13]:
print("Exporting...")
df.to_excel("test.xlsx", sheet_name = sheetName)
print("Exporting done!")

Exporting...
Exporting done!
